# Chapter 1, problem 16

We have a set $S$ with $n$ items and each item carries a value and a weight. Programmatically, we are talking about a collection of objects like
```python
class item:
    def __init__(self, value, weight):
        self.value = value
        self.weight = weight
```
A simpler way to represent these items may be with two syncronized arrays as the problem suggests: $S[1\ldots n]$ for the value of the items and $W[1\ldots n]$ for their weights. In this representation, $S[i]$ is the value of the $i$-th item and $W[i]$ its weight.

The problem asks us to identify an element $x\in S$ that splits the values into two halves: a subset $S_{<x}$ with elements whose values are less than $x$, and a subset $S_{>x}$ with elements whose values are greater than $x$. But with one constraint: the weight of the elements in $S_{<x}$ is less than or equal to half the total weight of all elements in $S$. And the weight of the elements in $S_{>x}$ is greater than or equal to half the total weight of all elements in $S$.

The problem is a modification of the [quickselect](https://en.wikipedia.org/wiki/Quickselect) algorithm. Quickselect finds the $k$-th smallest element in an array.

For this problem we need to compute first the half total weight of all elements in $S$. This is a constant value throughout the iterations:
$$ H = \frac{1}{2}\sum_{i=1}^n W[i] $$

The idea is to split the elements to a left and right groups by value around a pivot element. Then compute the weight of one of the groups - let's go for the left one. Determine where the half value would lie (left or right halves) and recurse.

This is best illustrated with an example using the following value and weight arrays 

- `S = [40, 10, 70, 20, 50, 30, 60]`
- `W = [ 2,  4,  1,  6,  3,  2,  1]`

In this example, the total weight is 19, so the half weight is 9.5  We are looking for that element $x$ in $S$ for which
\begin{align*}
\sum_{\begin{array}{c}y\in S\\y<x\end{array}} w(x) & \leq 9.5,\ \text{and} \\ \\
\sum_{\begin{array}{c}y\in S\\y>x\end{array}} w(x) & \leq 9.5
\end{align*}

As we make decisions about spliting the items into left and right partitions, we need to remember the weight of a partition prior to the next splitting. For that we introduce an accumulator, $A$ to track left partition weights and initialize it to zero.

**Step 1 — choose pivot `m = 40`**
- `L = {10, 20, 30}` with `w(L) = 4 + 6 + 2 = 12`
- `w(m) = 2`
- Check: `A + w(L) = 12 > 9.5` ⇒ **go left**; recurse on `L` with `A = 0`.

**Step 2 — on `S' = {10, 20, 30}` choose pivot `m = 20`**
- `L' = {10}` with `w(L') = 4`
- `w(m) = 6`
- Check: `A + w(L') = 4 <= 9.5` **and** `A + w(L') + w(m) = 10 >= 9.5` ⇒ **return `20`**.

**Verification**
- Weight left of `20`: `4 <= 9.5`
- Weight right of `20`: `19 - (4 + 6) = 9 <= 9.5`

Broadly, the algorithm can be described as:

\begin{align*}
& \textbf{weighted median}(S, W, A, H): \\
& \quad \textbf{if}\ |S|=1: \textbf{return}\ S[0] \\
& \quad m \leftarrow \text{pivot element selected from}\ S \\
& \quad L \leftarrow \{y: y\in S\ \text{and}\ y < m\} \\
& \quad W_L \leftarrow \sum_{z\in L} w_z\\
& \quad R \leftarrow \{y: y\in S\ \text{and}\ y > m\} \\
& \quad \textbf{if}\ A+W_L \leq H\ \text{and}\ A+W_L+w_m \geq H: \\
& \quad \quad \textbf{return}\ m\ {\color{gray}\texttt{ \# }\textsf{median found}}\\
& \quad \textbf{else if:}\  A+W_L > H: \\
& \quad \quad \textbf{return}\ \text{weighted median}(L, W, A, H)\  {\color{gray}\texttt{ \# }\textsf{recurse to the left}}\\
& \quad \textbf{else:} \\
& \quad \quad \textbf{return}\ \text{weighted median}(R, W, A+W_L+w_m, H)\  {\color{gray}\texttt{ \# }\textsf{recurse to the right}}
\end{align*}

with initial call
\begin{align*}
& H \leftarrow 0 \\
& \textbf{for}\ i\leftarrow 0\ldots n-1 \\
& \quad H \leftarrow H+W[i]\\
& \textbf{weighted median}(S, W, {\color{red}0,} H):
\end{align*}


# Chapter 1, problem 33

Given the example in the problem:

```
a = [9 13 16 18 19 23 28 31 37 42 1 3 4 5 7 8]
                              ↑               
```
where the arrow points to the rotation point, we may be tempted to find the location $k$ through a linear search:
\begin{align*}
& \textbf{find rotation point in array}(a): \\
& \quad k \leftarrow 1 \\
& \quad \textbf{while}\ k < \text{len}(a)\ \text{and}]\ a[k-1] \leq a[k]:\\
& \quad\quad k \leftarrow k+1 \\
& \quad\textbf{return}\ k
\end{align*}

This search runs in $\mathcal{O}(n)$ which is not bad but we can do better. What can be better than linear time? Ideally fixed time $\mathcal{O}(1)$ but realistically we can try for logarithmic time $\mathcal{O}(\log n)$ using binary search. This is based on the observation that an initially sorted array that has been rotated $k$ times has one half that is not distorted. In the example above, the left half of $a$ is perfectly sorted, and this can be determined by the test:

\begin{align*}
a[0] \leq a[\frac{n}{2}-1]: 9 < 28 &\ {\color{blue}\text{true}} \\
a[\frac{n}{2}] < a[n-1]: 31 < 8 &\ {\color{red}\text{false}}
\end{align*}

This observation is exploited in the pseudocode below

\begin{align*}
& \textbf{find rotation point in array}(a): \\
& \quad \text{low} \leftarrow 0 \\
& \quad \text{high} \leftarrow \text{len}(a)-1 \\
& \quad \textbf{while}\ \text{low} < \text{high} \\
& \quad \quad \text{mid}\leftarrow \lfloor (\text{low} - \text{high})/2 \rfloor \\
& \quad \quad \textbf{if}\ a[\text{mid}] > a[\text{high}]: \\
& \quad \quad \text{low}\leftarrow\text{mid}+1\ & {\color{gray}\texttt{ \# }\textsf{minimum is to the right of mid}} \\
& \quad \textbf{else} \\
& \quad \quad \text{high}\leftarrow\text{mid}\ & {\color{gray}\texttt{ \# }\textsf{minimum is at or left of mid}} \\
& \quad \text{mid}\leftarrow\text{low} & {\color{gray}\texttt{ \# }\textsf{index of minimum}} \\
& \quad\textbf{return}\ m-1  & {\color{gray}\texttt{ \# }\ k}
\end{align*}

The second part of this problem is a membership test for some value $x$: is $x$ present in array $a$ or not? Again, the idea is to split the array into two halves first and find which half is the sorted one. Check if $x$ falls within the range of that half (the range been the first and last element of the half). If so, recurse on the sorted half, otherwise on the other half.